In [12]:
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scipy.cluster import hierarchy
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.datasets import load_breast_cancer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import shap
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm

import lime
import lime.lime_tabular
from sklearn.metrics import f1_score

In [2]:
random_seed = 10

In [3]:
np.random.seed(random_seed)

In [81]:
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=random_seed)

In [5]:
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

In [82]:
lreg = LogisticRegression(random_state=0)
lreg.fit(X_train, y_train)

nbayes_g = GaussianNB()
nbayes_g.fit(X_train, y_train)

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


GaussianNB(priors=None, var_smoothing=1e-09)

In [83]:
pred_lreg = lreg.predict(X_test_transformed)
print(f1_score(y_test, pred_lreg, average='binary'))

pred_nbayes = nbayes_g.predict(X_test_transformed)
print(f1_score(y_test, pred_nbayes, average='binary'))

0.5542168674698795
0.02173913043478261


### LIME Explanations

In [10]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=data.feature_names, class_names=data.target_names, verbose=False, mode='classification',discretize_continuous=False)

In [18]:
explained_class = 1

In [79]:
lime_exp_nbayes = []
for i in range(X_train.shape[0]):
    exp_nbayes = explainer.explain_instance(X_train[i], nbayes.predict_proba, num_features=30)
    lime_exp_nbayes.append(exp_nbayes.as_list())

In [80]:
lime_exp_nbayes[4]

[('worst area', -0.3824415494633563),
 ('area error', -0.10666885793585361),
 ('mean area', 0.06874552450128696),
 ('worst perimeter', 0.050870180633143636),
 ('mean perimeter', 0.04060582037565526),
 ('worst texture', 0.016314343139973483),
 ('worst radius', 0.012016305470286606),
 ('mean texture', 0.01161256298572926),
 ('mean radius', 0.010640475269052251),
 ('worst concavity', 0.009687102568035547),
 ('mean concavity', 0.00821545610230401),
 ('mean symmetry', 0.006858295170994761),
 ('worst fractal dimension', -0.006694978955510569),
 ('smoothness error', 0.0054844191522322545),
 ('compactness error', -0.005187128674030104),
 ('worst concave points', -0.005075118189509526),
 ('mean concave points', -0.004696983870078934),
 ('perimeter error', 0.00449362563696409),
 ('mean smoothness', -0.004432306278461145),
 ('worst compactness', 0.003803861322427776),
 ('concavity error', -0.0030218906775774457),
 ('radius error', -0.0024858335924041065),
 ('fractal dimension error', -0.0014711492037498225),
 ('concave points error', 0.0014295286857398894),
 ('mean compactness', 0.001273116410230217),
 ('symmetry error', -0.0008122658129719611),
 ('texture error', 0.0007648112578328007),
 ('worst smoothness', 0.0006034697674587832),
 ('mean fractal dimension', 0.00042815882608581937),
 ('worst symmetry', 2.6595662100811225e-05)]

In [44]:
#lime_exp_nbayes = np.array(lime_exp_nbayes)

In [61]:
np.argwhere(lime_exp_nbayes[0][0][0] == data.feature_names)[0]

array([0])

In [68]:
def transform_lime_exp(exps, features):
    transform_exps = np.zeros((len(exps), len(features)))
    for i in range(len(exps)):
        for j in range(len(exps[i])):
            idx = np.argwhere(exps[i][j][0] == features)[0][0]
            transform_exps[i][idx] = exps[i][j][1]
    return transform_exps

In [69]:
transform_lime_bayes = transform_lime_exp(lime_exp_nbayes, data.feature_names)

In [72]:
transform_lime_bayes[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [73]:
lime_exp_nbayes[1]

[('mean radius', 0.0),
 ('mean texture', 0.0),
 ('mean perimeter', 0.0),
 ('mean area', 0.0),
 ('mean smoothness', 0.0),
 ('mean compactness', 0.0),
 ('mean concavity', 0.0),
 ('mean concave points', 0.0),
 ('mean symmetry', 0.0),
 ('mean fractal dimension', 0.0),
 ('radius error', 0.0),
 ('texture error', 0.0),
 ('perimeter error', 0.0),
 ('area error', 0.0),
 ('smoothness error', 0.0),
 ('compactness error', 0.0),
 ('concavity error', 0.0),
 ('concave points error', 0.0),
 ('symmetry error', 0.0),
 ('fractal dimension error', 0.0),
 ('worst radius', 0.0),
 ('worst texture', 0.0),
 ('worst perimeter', 0.0),
 ('worst area', 0.0),
 ('worst smoothness', 0.0),
 ('worst compactness', 0.0),
 ('worst concavity', 0.0),
 ('worst concave points', 0.0),
 ('worst symmetry', 0.0),
 ('worst fractal dimension', 0.0)]